In [128]:
import os
import requests
import json
import pandas as pd
from mysql import connector
from dotenv import load_dotenv
import numpy as np
from datetime import datetime

In [129]:
coins = ['bitcoin', 'solana']

load_dotenv()

url_base = "https://api.coingecko.com/api/v3/coins/"
days = 30
params = {
    "vs_currency": "usd",
    "days": days
}

print(params)

{'vs_currency': 'usd', 'days': 30}


# EXTRACT

In [130]:
payloads = []


for idx, coin in enumerate(coins):
    url = f"{url_base}{coin}/ohlc"
    response = requests.get(url, params=params)
    payloads.append(response.json())
    
formatted_response = json.dumps(payloads, indent=4)


# TRASNFORM

In [131]:
ohlc_df1 = pd.DataFrame(payloads)
ohlc_df1

,0,1,2,3,4,5,6,7,8,9,...,170,171,172,173,174,175,176,177,178,179
0,"[1755979200000, 114783.0, 115129.0, 114707.0, ...","[1755993600000, 115099.0, 115360.0, 114983.0, ...","[1756008000000, 115333.0, 115549.0, 114909.0, ...","[1756022400000, 114993.0, 115199.0, 114845.0, ...","[1756036800000, 114859.0, 114927.0, 114575.0, ...","[1756051200000, 114678.0, 114802.0, 114390.0, ...","[1756065600000, 114370.0, 114741.0, 112217.0, ...","[1756080000000, 112523.0, 113617.0, 112174.0, ...","[1756094400000, 113423.0, 113574.0, 112253.0, ...","[1756108800000, 112971.0, 112971.0, 111501.0, ...",...,"[1758427200000, 115723.0, 115768.0, 115527.0, ...","[1758441600000, 115702.0, 115850.0, 115539.0, ...","[1758456000000, 115858.0, 115858.0, 115426.0, ...","[1758470400000, 115680.0, 115796.0, 115529.0, ...","[1758484800000, 115592.0, 115674.0, 115309.0, ...","[1758499200000, 115525.0, 115585.0, 115257.0, ...","[1758513600000, 115256.0, 115394.0, 114269.0, ...","[1758528000000, 114694.0, 114694.0, 112593.0, ...","[1758542400000, 112586.0, 112978.0, 112293.0, ...","[1758556800000, 112855.0, 113384.0, 112604.0, ..."
1,"[1755979200000, 202.53, 204.19, 202.44, 204.06]","[1755993600000, 204.02, 204.67, 202.94, 204.02]","[1756008000000, 204.27, 210.85, 204.27, 208.66]","[1756022400000, 208.95, 210.08, 207.08, 207.08]","[1756036800000, 206.8, 208.47, 203.88, 204.41]","[1756051200000, 204.5, 208.6, 203.77, 206.07]","[1756065600000, 205.85, 211.54, 203.89, 203.89]","[1756080000000, 204.23, 206.72, 202.06, 205.88]","[1756094400000, 205.89, 212.69, 203.3, 210.35]","[1756108800000, 209.67, 209.67, 199.76, 199.76]",...,"[1758427200000, 239.56, 241.47, 239.31, 241.47]","[1758441600000, 241.69, 241.69, 239.88, 240.11]","[1758456000000, 240.05, 240.05, 237.54, 238.86]","[1758470400000, 239.04, 240.05, 238.04, 238.19]","[1758484800000, 238.13, 238.33, 236.62, 237.89]","[1758499200000, 237.94, 238.03, 235.76, 236.65]","[1758513600000, 236.45, 236.69, 231.24, 232.82]","[1758528000000, 232.82, 232.92, 221.91, 222.38]","[1758542400000, 222.03, 222.65, 220.28, 221.75]","[1758556800000, 221.5, 223.83, 220.51, 220.51]"


In [132]:
rows = []

for idx, coin in enumerate(coins):
    total_records = len(payloads[idx])

    for i in range(total_records):
        timestamp_ms = payloads[idx][i][0]
        timestamp = datetime.fromtimestamp(timestamp_ms / 1000)

        open = round(payloads[idx][i][1], 5)
        high = round(payloads[idx][i][2], 5)
        low = round(payloads[idx][i][3], 5)
        close = round(payloads[idx][i][4], 5)

        row = {
            'coin_id': coin,
            'timestamp': timestamp,
            'open_price': open,
            'high_price': high,
            'low_price': low,
            'close_price': close
        }
        rows.append(row)

ohlc_df = pd.DataFrame(rows)
ohlc_df


,coin_id,timestamp,open_price,high_price,low_price,close_price
0,bitcoin,2025-08-23 16:00:00,114783.00,115129.00,114707.00,115092.00
1,bitcoin,2025-08-23 20:00:00,115099.00,115360.00,114983.00,115360.00
2,bitcoin,2025-08-24 00:00:00,115333.00,115549.00,114909.00,115005.00
3,bitcoin,2025-08-24 04:00:00,114993.00,115199.00,114845.00,114880.00
4,bitcoin,2025-08-24 08:00:00,114859.00,114927.00,114575.00,114695.00
...,...,...,...,...,...,...
355,solana,2025-09-21 20:00:00,237.94,238.03,235.76,236.65
356,solana,2025-09-22 00:00:00,236.45,236.69,231.24,232.82
357,solana,2025-09-22 04:00:00,232.82,232.92,221.91,222.38
358,solana,2025-09-22 08:00:00,222.03,222.65,220.28,221.75


# LOAD

In [133]:
MYSQL_USER = os.getenv("MYSQL_USER")
MYSQL_PASSWORD = os.getenv("MYSQL_PASSWORD")
MYSQL_HOST = os.getenv("MYSQL_HOST")
MYSQL_PORT = os.getenv("MYSQL_PORT")
MYSQL_DB = os.getenv("MYSQL_DATABASE")

In [134]:
db_conn = connector.connect(
    host=MYSQL_HOST,
    user=MYSQL_USER,
    password=MYSQL_PASSWORD,
    port=MYSQL_PORT,
    database=MYSQL_DB,
    connection_timeout=10,
    autocommit=False,
    raise_on_warnings=True
)   

db_cur = db_conn.cursor()
print(f"[SUCCESS] Connected to MySQL db {MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DB} as user {MYSQL_USER}")

[SUCCESS] Connected to MySQL db localhost:3306/cryptodb as user root


In [135]:
sql_table = "coin_ohlc"
db_cur.execute(f"SHOW TABLES LIKE '{sql_table}'")

if db_cur.fetchone() is None:
    raise SystemExit(f"[ERROR] Table '{sql_table}' does not exist in database '{MYSQL_DB}'")
else:
    print(f"[SUCCESS] Table '{sql_table}' exists in database '{MYSQL_DB}'")

[SUCCESS] Table 'coin_ohlc' exists in database 'cryptodb'


In [136]:
INSERT_SQL = f"""
INSERT INTO {sql_table} (
    coin_id, timestamp, open_price, high_price, low_price, close_price
) VALUES (
    %s, %s, %s, %s, %s, %s
)
"""

In [137]:
ohlc_list = ohlc_df.values.tolist()

In [138]:
try:
    db_cur.executemany(INSERT_SQL, ohlc_list)
    db_conn.commit()
    print(f"[SUCCESS] Inserted {db_cur.rowcount} records into table '{sql_table}'")
except connector.Error as err:
    db_conn.rollback()
    print(f"[ERROR] Failed to insert records into table '{sql_table}': {err}")
finally:
    db_cur.close()
    db_conn.close()
    print("[INFO] MySQL connection closed")

[SUCCESS] Inserted 360 records into table 'coin_ohlc'
[INFO] MySQL connection closed
